## Swarm team
Is a team inspired by the one [provided](https://github.com/openai/swarm) by OpenAI where an agent can decide to hand off a task to another agent.  

It is basically a group chat where,like the ones seen before, each agent broadcasts their responses to all the other agents in the group but the next one depends
on the most recent `HandsoffMessage`

**NOTE** The handoff functionality uses the tool calling functionality, this means that the LLM that the agent uses **MUST** support tool calling.

In [7]:
import sys
sys.path.append("..")

from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.teams import Swarm
from autogen_agentchat.messages import HandoffMessage
from autogen_agentchat.ui import Console
from model_clients.azure import get_model
from rich import print


model_client = get_model()

As usual, let's now define the tools involved

In [8]:
def refund_flight(flight_id: str) -> str:
    """Refund a flight

    :param flight_id: The ID of the flight.
    :return: The confirmation that the flight has been refunded.
    """
    print(f"[blue]Refunding flight {flight_id}[/blue]")
    return f"Flight {flight_id} refunded"


Now its time to define the agents...

In [9]:
travel_agent = AssistantAgent(
    "travel_agent",
    model_client=model_client,
    handoffs=["flights_refunder", "user"],
    system_message="""You are a travel agent.
    The flights_refunder is in charge of refunding flights.
    If you need information from the user, you can handoff to the user.
    Use TERMINATE when the travel planning is complete.""",
)

flights_refunder = AssistantAgent(
    "flights_refunder",
    model_client=model_client,
    handoffs=["travel_agent", "user"],
    tools=[refund_flight],
    system_message="""You are an agent specialized in refunding flights.
    You only need flight reference numbers to refund a flight.
    You have the ability to refund a flight using the refund_flight tool.
    If you need information from the user, you must first send your message, then you can handoff to the user.
    When the transaction is complete, handoff to the travel agent to finalize.
    Use TERMINATE when the refunding is complete or when the bookins id is 'exit'.""",
)

# user_agent= UserProxyAgent(name="user",
#                             description= "The user in charge of handling additional requests from the assistant",input_func=input)

And finally the Swarm team with the proper termination conditions

In [10]:
termination = HandoffTermination(target="user") | TextMentionTermination("TERMINATE")
team = Swarm([travel_agent, flights_refunder], termination_condition=termination)


Now let's run the workflow, you can see the handsoff in action using, as example:  

- 'I need help with my flight'
- 'I need a refund'
- 'I want my flight 678 refunded'

And test the different behavior

In [11]:

# This is the task that the user wants to accomplish.
task = input("How can the agent help you?: ")

# Run the task
task_result = await Console(team.run_stream(task=task))

print(task_result.messages)
# Get the last message because it might be a handoff message to the user
last_message = task_result.messages[-1]

while isinstance(last_message, HandoffMessage) and last_message.target == "user":
    user_message = input("User: ")
    # Note that the user message is sent to the last agent that handed off to the user (via source)
    # And since the Team keeps its state, the previous information is still available.
    task_result = await Console(
        team.run_stream(task=HandoffMessage(source="user", target=last_message.source, content=user_message))
    )
    last_message = task_result.messages[-1]

---------- user ----------
I need help with my flight
---------- travel_agent ----------
Could you please provide more details about what you need help with regarding your flight? Are you looking to book a new flight, change an existing reservation, or request a refund?
---------- travel_agent ----------
[FunctionCall(id='call_kmfYcLZe0np9UOimdCNRkwcT', arguments='{}', name='transfer_to_user')]
---------- travel_agent ----------
[FunctionExecutionResult(content='Transferred to user, adopting the role of user immediately.', name='transfer_to_user', call_id='call_kmfYcLZe0np9UOimdCNRkwcT', is_error=False)]
---------- travel_agent ----------
Transferred to user, adopting the role of user immediately.


[
    TextMessage(
        source='user',
        models_usage=None,
        metadata={},
        content='I need help with my flight',
        type='TextMessage'
    ),
    TextMessage(
        source='travel_agent',
        models_usage=RequestUsage(prompt_tokens=110, completion_tokens=37),
        metadata={},
        content='Could you please provide more details about what you need help with regarding your flight? Are you
looking to book a new flight, change an existing reservation, or request a refund?',
        type='TextMessage'
    ),
    ToolCallRequestEvent(
        source='travel_agent',
        models_usage=RequestUsage(prompt_tokens=152, completion_tokens=12),
        metadata={},
        content=[FunctionCall(id='call_kmfYcLZe0np9UOimdCNRkwcT', arguments='{}', name='transfer_to_user')],
        type='ToolCallRequestEvent'
    ),
    ToolCallExecutionEvent(
        source='travel_agent',
        models_usage=None,
        metadata={},
        content=[
            FunctionExecutionResult(
                content='Transferred to user, adopting the role of user immediately.',
                name='transfer_to_user',
                call_id='call_kmfYcLZe0np9UOimdCNRkwcT',
                is_error=False
            )
        ],
        type='ToolCallExecutionEvent'
    ),
    HandoffMessage(
        source='travel_agent',
        models_usage=None,
        metadata={},
        target='user',
        content='Transferred to user, adopting the role of user immediately.',
        context=[],
        type='HandoffMessage'
    )
]

---------- user ----------
a refund
---------- travel_agent ----------
[FunctionCall(id='call_XErLXmle1gMzSVcGpubvozVI', arguments='{}', name='transfer_to_flights_refunder')]
---------- travel_agent ----------
[FunctionExecutionResult(content='Transferred to flights_refunder, adopting the role of flights_refunder immediately.', name='transfer_to_flights_refunder', call_id='call_XErLXmle1gMzSVcGpubvozVI', is_error=False)]
---------- travel_agent ----------
Transferred to flights_refunder, adopting the role of flights_refunder immediately.
---------- flights_refunder ----------
Could you please provide the flight reference number for the flight you would like to refund?
---------- flights_refunder ----------
[FunctionCall(id='call_7w9FnbOQwCe8udXJhnXcXbxs', arguments='{}', name='transfer_to_user')]
---------- flights_refunder ----------
[FunctionExecutionResult(content='Transferred to user, adopting the role of user immediately.', name='transfer_to_user', call_id='call_7w9FnbOQwCe8udXJhn

Refunding flight 932

---------- flights_refunder ----------
[FunctionCall(id='call_HOXCQcYN8GBHNWjTIxZq3vMS', arguments='{"flight_id":"932"}', name='refund_flight')]
---------- flights_refunder ----------
[FunctionExecutionResult(content='Flight 932 refunded', name='refund_flight', call_id='call_HOXCQcYN8GBHNWjTIxZq3vMS', is_error=False)]
---------- flights_refunder ----------
Flight 932 refunded
---------- flights_refunder ----------
[FunctionCall(id='call_YNmB4FXZcbEAUncAXx0Tm7am', arguments='{}', name='transfer_to_travel_agent')]
---------- flights_refunder ----------
[FunctionExecutionResult(content='Transferred to travel_agent, adopting the role of travel_agent immediately.', name='transfer_to_travel_agent', call_id='call_YNmB4FXZcbEAUncAXx0Tm7am', is_error=False)]
---------- flights_refunder ----------
Transferred to travel_agent, adopting the role of travel_agent immediately.
---------- travel_agent ----------
Your refund request for flight 932 has been processed successfully. If you need further as